In [9]:
from PIL import Image
import os
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import numpy as np
import pandas as pd
from IPython.display import display, Markdown
import SimpleITK as sitk

import sys
sys.path.append('../../rse-panimg')

from pathlib import Path
from panimg import convert

In [10]:
ATTENTION_PARENT_DIR = f"V:/experiments/lung-malignancy-fairness-shaurya/nlst/sybil_attentions_fp"
ATTENTION_GIF_DIR_DIR = f"{ATTENTION_PARENT_DIR}/attention_gifs"
ATTENTION_PNG_DIR_DIR = f"{ATTENTION_PARENT_DIR}/attention_imgs"
ATTENTION_MHA_DIR_DIR = f"{ATTENTION_PARENT_DIR}/attention_mha2d"
ATTENTION_MHA3D_DIR   = f"{ATTENTION_PARENT_DIR}/attention_mha3d"

In [11]:
os.makedirs(ATTENTION_PNG_DIR_DIR, exist_ok=True)
os.makedirs(ATTENTION_MHA_DIR_DIR, exist_ok=True)
os.makedirs(ATTENTION_MHA3D_DIR, exist_ok=True)

In [12]:
subfolders = [f[:-4] for f in os.listdir(ATTENTION_GIF_DIR_DIR)]
subfolders

['serie_1.2.840.113654.2.55.101581432986749230362000431122230027626',
 'serie_1.2.840.113654.2.55.107286458984147654450019349189159324228',
 'serie_1.2.840.113654.2.55.121550455418938981371481534296630834431',
 'serie_1.2.840.113654.2.55.122632601354468061472344333696896368930',
 'serie_1.2.840.113654.2.55.124060744876084068113330751404318777416',
 'serie_1.2.840.113654.2.55.131668829951904754279077215563066892069',
 'serie_1.2.840.113654.2.55.135394718263679135674858017019222554025',
 'serie_1.2.840.113654.2.55.136304680579873018890306723085300841058',
 'serie_1.2.840.113654.2.55.142794431293514807907929705714113972574',
 'serie_1.2.840.113654.2.55.14310126728095959479975399446060032407',
 'serie_1.2.840.113654.2.55.148346971577860286371037732187617157208',
 'serie_1.2.840.113654.2.55.16240614331314545826126838738906285544',
 'serie_1.2.840.113654.2.55.173372354929362110329506024057282519541',
 'serie_1.2.840.113654.2.55.204202905668227766493937842163626937511',
 'serie_1.2.840.113654

In [13]:
# for i in range(1):
#     seriesid = subfolders[i]
#     n_frames = len(os.listdir(f"{ATTENTION_PNG_DIR_DIR}/{seriesid}"))
#     print(f"{seriesid}: {n_frames} frames")
#     result = convert(
#         input_directory=Path(f"{ATTENTION_PNG_DIR_DIR}/{seriesid}"),
#         output_directory=Path(f"{ATTENTION_MHA_DIR_DIR}/{seriesid}"),
#     )
#     print(result)

In [14]:
def split_gif_to_frames(gif_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    with Image.open(gif_path) as gif:
        frame_count = gif.n_frames
        print(f"Total frames: {frame_count}")

        for frame in range(frame_count):
            gif.seek(frame)
            frame_filename = os.path.join(output_folder, f"frame_{frame:03d}.png")
            gif.save(frame_filename, format="PNG")
            # print(f"Saved: {frame_filename}")

In [15]:
def stack_mha_files(input_folder, output_mha_path):
    # Get list of MHA files in the folder, sorted by filename (assuming sequential naming)
    frame_subdirs = sorted(os.listdir(input_folder))
    mha_files = [f'{sd}/{os.listdir(f"{input_folder}/{sd}")[0]}' for sd in frame_subdirs][1:]
    
    # mha_files = sorted([f for f in os.listdir(input_folder) if f.endswith('.mha')])

    if not mha_files:
        raise ValueError("No MHA files found in the provided directory")

    # Read the first MHA file to determine the size and spacing of the slices
    first_mha_path = os.path.join(input_folder, mha_files[0])
    first_image = sitk.ReadImage(first_mha_path, outputPixelType=sitk.sitkVectorUInt8)
    slice_size = first_image.GetSize()  # (width, height, depth)
    print("slice size:", slice_size, ", num_components:", first_image.GetNumberOfComponentsPerPixel())

    # Create an empty list to hold the images
    image_list = []

    # Load each MHA file and append the images to the list
    for i, mha_file in enumerate(mha_files):
        mha_path = os.path.join(input_folder, mha_file)
        img = sitk.ReadImage(mha_path, sitk.sitkVectorUInt8)
        # print("slice ", i, " size:", img.GetSize(), ", num_components:", img.GetNumberOfComponentsPerPixel())
        if img.GetNumberOfComponentsPerPixel() == 3:
            image_list.append(img)

    # Stack the images along the third dimension (Z-axis)
    print("number of frames in stack:", len(image_list))
    stacked_image = sitk.JoinSeries(image_list)
    stack_size = stacked_image.GetSize()
    print("stack size:", stack_size, ", stack n_components:", stacked_image.GetNumberOfComponentsPerPixel())

    # Save the stacked image as a single 3D MHA file
    sitk.WriteImage(stacked_image, output_mha_path)
    print(f"Stacked MHA saved as {output_mha_path}")

In [16]:
# for i in range(1):
#     seriesid = subfolders[i]
#     n_frames = len(os.listdir(f"{ATTENTION_MHA_DIR_DIR}/{seriesid}"))
#     print(f"{seriesid}: {n_frames} subfolders")
#     input_folder = f"{ATTENTION_MHA_DIR_DIR}/{seriesid}"  # Folder containing the individual MHA slices
#     output_mha_path = f"{ATTENTION_MHA3D_DIR}/{seriesid}.mha"  # Path to save the stacked MHA file
#     stack_mha_files(input_folder, output_mha_path)

In [19]:
for i in range(44, len(subfolders)):
    seriesid = subfolders[i]
    gif_path = f"{ATTENTION_GIF_DIR_DIR}/{seriesid}.gif"
    output_folder = f"{ATTENTION_PNG_DIR_DIR}/{seriesid}"
    print(f"\n{i+1} / {len(subfolders)}: converting {seriesid} ...")
    split_gif_to_frames(gif_path, output_folder)

    n_frames = len(os.listdir(f"{ATTENTION_PNG_DIR_DIR}/{seriesid}"))
    print(f"{n_frames} frames")
    result = convert(
        input_directory=Path(f"{ATTENTION_PNG_DIR_DIR}/{seriesid}"),
        output_directory=Path(f"{ATTENTION_MHA_DIR_DIR}/{seriesid}"),
    )
    print(result)

    n_frames = len(os.listdir(f"{ATTENTION_MHA_DIR_DIR}/{seriesid}"))
    print(f"{n_frames} subfolders")
    input_folder = f"{ATTENTION_MHA_DIR_DIR}/{seriesid}"  # Folder containing the individual MHA slices
    output_mha_path = f"{ATTENTION_MHA3D_DIR}/{seriesid}.mha"  # Path to save the stacked MHA file
    stack_mha_files(input_folder, output_mha_path)


45 / 50: converting serie_1.3.6.1.4.1.14519.5.2.1.7009.9004.105222755865394382014047245157 ...
Total frames: 197
197 frames
PanImgResult(new_images={PanImg(pk=UUID('52cd8e4f-408a-46d5-801b-736debec10b7'), name='frame_029.png', width=512, height=512, depth=None, voxel_width_mm=None, voxel_height_mm=None, voxel_depth_mm=None, timepoints=None, resolution_levels=None, window_center=None, window_width=None, color_space=<ColorSpace.RGB: 'RGB'>, eye_choice=<EyeChoice.NOT_APPLICABLE: 'NA'>, patient_id='', patient_name='', patient_birth_date=None, patient_age='', patient_sex=<PatientSex.EMPTY: ''>, study_date=None, study_instance_uid='', series_instance_uid='', study_description='', series_description='', segments=None), PanImg(pk=UUID('ef42e541-b071-4c98-9d4d-c2119caf2de4'), name='frame_181.png', width=512, height=512, depth=None, voxel_width_mm=None, voxel_height_mm=None, voxel_depth_mm=None, timepoints=None, resolution_levels=None, window_center=None, window_width=None, color_space=<ColorSp